<a href="https://colab.research.google.com/github/con11235/SWCON424_FDA_Project/blob/main/MV-model/MV_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mean-Variance Model
본 ipynb는 colab을 사용하여 작성하였습니다.<br>
생성된 주식 파일의 저장 위치는 drive를 마운트하여 '/content/drive/MyDrive/FDA/'의 위치에 저장하였습니다.<br>
파일이 필요한 경우 [공유파일](https://drive.google.com/file/d/1-5WAFWZy7LsHps72jH6HgVMar5MyW7WU/view?usp=sharing)으로 다운 가능합니다.<br>
파일을 생성하는 코드는 [github](https://github.com/con11235/SWCON424_FDA_Project/blob/main/input_data_save.ipynb)에서 확인 가능합니다.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## import module

In [9]:
import pandas as pd
import numpy as np
import cvxpy as cp
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm

아래는 이후 나오는 값들을 보기 용이하게 하고자 과학적 표기법을 사용하지 않도록 하는 코드입니다.

In [10]:
np.set_printoptions(suppress = True)

## Load raw data

저장된 파일을 불러옵니다.

In [11]:
all_df = pd.read_csv('/content/drive/MyDrive/인연준/금융데이터분석/input/all_asset_2010_2020.csv')
all_df['Date'] = all_df.Date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
all_df.set_index('Date',inplace=True)

## User input

사용자로부터 직접 input을 받을 수 있는 코드입니다.<br>
st와 en의 경우 년-월-일 의 형태로 입력받습니다.<br>
unittime과 lookback, rebalance는 년, 월, 주, 일 단위를 선택할 수 있으며, 각 단위에 대한 기간을 숫자로 입력합니다.<br>
예를 들어, 3년(3y), 6개월(6m), 5주(5w), 1일(1d)의 단위로 입력합니다.

In [12]:
# symbols = []
# while True:
#     name = input("종목코드 : ")
#     if name == '0':
#         break
#     symbols.append(name)
# st = input("시작일 0000-00-00: ")
# en = input("종료일 0000-00-00: ")
# unittime = input("투자 단위기간 : ")
# lookback = input("Look back : ")
# rebalance = input("Rebalance : ")

코드 실행을 위해 임의로 input을 대입한 값입니다.<br>
symbols는 평가하고자 하는 주식의 코드들을 의미합니다.<br>
st는 투자를 시작할 날짜를 의미합니다.<br>
en는 투자를 종료할 날짜를 의미합니다.<br>
unittime은 backtest에서 확인할 포트폴리오의 단위기간입니다.<br>
lookback은 backtest에서 사용할 lookback의 기간입니다.<br>
rebalance는 backtest에서 rebalancing할 기간의 단위입니다.<br>


In [13]:
symbols = ['AAPL', 'MSFT', 'AMZN', 'FB']#, 'GOOGL', 'GOOG', 'TSLA', 'NVDA', 'CMCSA', 'ADBE']
st ='2020-06-01'
en = '2020-10-30'
unittime = '1m'
lookback = '6m'
rebalance = '2m'

이후 날짜 설정을 용이하게 하기 위해 datetime으로 변경하는 부분입니다.

In [14]:
st = datetime(int(st[:4]),int(st[5:7]),int(st[8:10]))
en = datetime(int(en[:4]),int(en[5:7]),int(en[8:10]))
def timestr_to_datetime(s):
  if s[-1] == 'd':
    return timedelta(days=int(s[:-1]))
  elif s[-1] == 'w':
    return timedelta(weeks=int(s[:-1]))
  elif s[-1] == 'm':
    return relativedelta(months=int(s[:-1]))
  elif s[-1] == 'y':
    return relativedelta(years=int(s[:-1]))

unittime = timestr_to_datetime(unittime)
lookback = timestr_to_datetime(lookback)
rebalance = timestr_to_datetime(rebalance)

## Define of Mean-Variance model

분석에 사용할 data를 받아 포트폴리오의 분산을 계산하고, cvxpy를 사용하여 minimize합니다.<br>
이때 들어가는 data는 단위 기간에 맞춰 필요한 값만 가진 data이어야 합니다.<br>
예를 들어 2018년 9월부터 2019년 9월의 데이터를 가지고 2달 단위로 계산한 결과를 얻고싶다면 데이터는 18/9, 18/11, 19/1, 19/3, 19/5, 19/7, 19/9의 데이터만을 가지고있어야 합니다.<br>
input의 cons로 원하는 제약조건을 리스트 형태로 입력받을 수 있습니다.<br>
제약조건의 순서는 상관없지만, 원하는 제약조건의 명칭을 입력해야 제약조건을 사용할 수 있습니다.<br>
제약조건의 종류는 다음과 같습니다.<br>
* 'n-sh' : No Shorting
* 'tr-c' : Transaction Cost

In [15]:
data = all_df[symbols]
data_ret = data.pct_change().dropna()

In [16]:
import pandas_datareader.data as web
ff3 = web.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2010-01-01', end='2020-11-01')
ff3 = ff3[0]/100

f_df = data_ret.copy()
for a in ff3.columns:
  f_df[a] = ff3[a]

In [18]:
def regression(df,symbols):
  temp = pd.DataFrame()
  for sym in symbols:
    Y = df[sym]-df['RF']
    X = sm.add_constant(df[['Mkt-RF', 'SMB', 'HML']])
    model = sm.OLS(Y, X)
    results = model.fit()
    temp[sym] = results.params
  
  return temp.iloc[1:,:]

In [28]:
def mv_opt(data,fama=None,ret_target = None,cons=[''], effect_of_factor = None):
  n = len(data.T)   # asset number
  W = cp.Variable(n)  # weight of assets
  R = data.mean()*252
  C = data.cov()*252
  constraints = [cp.sum(W) == 1]
  if ret_target != None:
    ret_gvm = np.dot(R.T, mv_opt(data,fama=None)/100)
    ret_target = np.round(ret_gvm,2) + ret_target
  else:
    ret_target = 0
  
  constraints.append(W @ R >= ret_target)
  objective = cp.Minimize(cp.quad_form(W, C))
  solver = cp.ECOS

  if 'n-sh' in cons:
    constraints.append(W >= 0)
  
  if 'risk-factor' in cons:
    if effect_of_factor is None:
      effect_of_factor = 10**8
    beta = regression(fama, symbols)
    constraints.append(beta.values @ W <= effect_of_factor)

  problem = cp.Problem(objective, constraints)
  problem.solve(solver=solver)

  return W.value.round(6)*100

1.개별 자산의 turnover 비율 설정

$|w_i - w_{0,i}| \leq u_i~~~for~i = 1,...,n$

2.전체 자산의 총 turnover 비율 설정

$\sum_i{|w_i - w_{0,i}|} \leq u$




In [32]:
mv_opt(data_ret,fama=f_df, ret_target = 0, cons=['n-sh','risk-factor'], effect_of_factor = 10)

array([26.9934, 38.8818, 24.2939,  9.8308])

## Define of Backtest

backtest를 진행하는 함수입니다.<br>
원하는 기간과 단위 기간에 맞는 데이터를 가져온 후, pct_change 값을 계산합니다.<br>
결과는 각 종목별 amount, 전체 amount, 포트폴리오의 return, 각 종목별 weight 순으로 반환합니다.<br>
dropna는 기본값으로 True로 되어있는데 이 경우 모든 기간동안 weight가 0이었던 종목은 제외하고 결과를 보여줍니다.<br>
이후 rebalancing 기간마다 weight를 계산하고, 단위 기간마다 투자 결과를 보여주는 표를 결과로 반환합니다.<br>

In [ ]:
def backtest(symbols, st, en, ut,lb, rb,dropna=True):
  def data_by_dates():
    data = all_df[symbols]
    df = pd.DataFrame(columns = symbols)
    date = st-lookback
    while date < en:
        df.loc[data[date:].index[0]] = data[date:].values[0]
        date += ut
    rb_date = []
    date = st
    while date < en:
        rb_date.append(data[date:].index[0])
        date += rb
    return df, rb_date
  df, rb_date = data_by_dates()
  df_ret = df.pct_change().dropna()
  
  col = [i+'_amt' for i in symbols]
  col.append('total_amt')
  col.append('portfolio_ret')
  col+= [i+'_weight' for i in symbols]
  result = pd.DataFrame(columns = col)
  T_amt = 100

  for index, row in df.iterrows():
    if st <= index:
      if index in rb_date:
        weight = mv_opt(df_ret[index-lb:index])
      amt = (df_ret[index:index]+1)*weight*T_amt/100
      for i in range(len(symbols)):
        s = symbols[i]
        result.loc[index,s+'_amt'] = amt[s].values[0]
        result.loc[index,s+'_weight'] = weight[i]
      T_amt_ = T_amt
      T_amt = amt.sum(axis = 1).values[0]
      result.loc[index,'total_amt'] = T_amt
      result.loc[index,'portfolio_ret'] = T_amt/T_amt_ - 1
  if dropna == True:
    return result.loc[:,result.eq(0).sum(axis=0) != len(result)]
  return result


In [ ]:
backtest(symbols, st, en, unittime, lookback, rebalance)

,MSFT_amt,AMZN_amt,NVDA_amt,CMCSA_amt,total_amt,portfolio_ret,MSFT_weight,AMZN_weight,NVDA_weight,CMCSA_weight
2020-06-01,0.69662,96.1314,12.9579,0,109.786,0.0978595,0.6633,88.9343,10.4024,0
2020-07-01,0.815318,113.745,12.3646,0,126.925,0.156114,0.6633,88.9343,10.4024,0
2020-08-03,6.43929,102.153,30.4307,0,139.023,0.0953172,4.7959,74.4521,20.752,0
2020-09-01,7.01471,116.385,36.2259,0,159.626,0.148198,4.7959,74.4521,20.752,0
2020-10-01,0,103.084,0,49.121,152.205,-0.0464933,0,70.1485,0,29.8515
